In [4]:
import requests
import pandas as pd
import time

# 행정동 코드 추가

In [2]:
# setting
SEARCH_CONFIRM_KEY = "devU01TX0FVVEgyMDI1MDkxODIzMzA1ODExNjIzMjI="
url = "https://business.juso.go.kr/addrlink/addrLinkApi.do"

In [74]:
# load data
data1 = pd.read_csv(r"C:\Users\USER\PycharmProjects\Big-Contest\data\raw\big_data_set1_f.csv", encoding="cp949")
행정동 = pd.read_csv(r"C:\Users\USER\PycharmProjects\Big-Contest\data\raw\행정동_변환_관련\통계청_법정동 연계정보_20250602.csv", encoding="euc-kr")
법정동 = pd.read_csv(r"C:\Users\USER\PycharmProjects\Big-Contest\data\raw\행정동_변환_관련\국토교통부_전국 법정동_20250807.csv", encoding='utf-8')

In [75]:
# prepare data
data1["MCT_BSE_AR"] = data1["MCT_BSE_AR"].str.split(n=2).str[2]
data1["MCT_BSE_AR"] = data1["MCT_BSE_AR"].str.rstrip('.')
data1["MCT_BSE_AR"] = data1["MCT_BSE_AR"].str.replace("  ", " ")
data1["MCT_BSE_AR"] = "서울특별시 성동구 "+data1["MCT_BSE_AR"]

서울_성동구_법정동 = 법정동[(법정동["시도명"]=="서울특별시") & (법정동["시군구명"]=="성동구") & (pd.isna(법정동["삭제일자"]))]
서울_성동구_법정동_행정동 = pd.merge(서울_성동구_법정동, 행정동, on="법정동코드")
서울_성동구_법정동_행정동.drop_duplicates(subset=["행정동코드"])
hjd_nm_to_code_dict = (
	서울_성동구_법정동_행정동[["행정동명", "행정동코드"]]
	.dropna()
	.drop_duplicates(subset=["행정동명"])
	.set_index("행정동명")["행정동코드"]
	.to_dict()
)

In [ ]:
# send request - timeout error 발생 -> checkpoint 
i = 0
doro_hjd_nm_dict = {}
no_results_doro = []
fail_doro = []

In [63]:
# send request 
for idx, addr in enumerate(pd.unique(data1["MCT_BSE_AR"])[i:], start=i):
	params = {"confmKey" : SEARCH_CONFIRM_KEY,
		  "currentPage" : 1,
		   "countPerPage" : 10,
		   "keyword" : addr,
		   "resultType" : "json",
		   "addInfoYn" : "Y"
		}
	try:
		response_json = requests.get(url, params=params, timeout=(5, 10)).json()
		if len(response_json["results"]["juso"]) > 1:
			print(i, "2개 이상의 결과가 존재함:", addr) 
			# 서울특별시 성동구 상원2길 5를 검색하면 서울특별시 성동구 상원1길 5-5, 5-6, .. 이런 식으로 더 나옴.
			# 이런 경우 맨 첫 번째 결과가 가장 적합한 결과가 나오는 듯함.
			first = response_json["results"]["juso"][0]
			doro_hjd_nm_dict[addr] = first["hemdNm"]
		elif len(response_json["results"]["juso"]) < 1:
			print(i, "검색 결과가 존재하지 않음: ", addr)
			# 업데이트 되지 않은 도로명주소일 경우 이렇게 나옴.
			# 따로 모은 뒤에 그 양에 따라 어떻게 할지 정해볼 듯함.
			# 가능한 방법 1) 도로명주소에 직접 검색->크롤링
			# 가능한 방법 2) 법정동 -> 행정동/혹은 바로 행정동 코드로 시도해보기
			# 가능한 방법 3) 도로명 주소 업데이트 api 찾아보기
			# 		== 매핑되지 않은 도로명 ==
			# ['서울특별시 성동구 옥수2동', '서울특별시 성동구 매봉길 13', '서울특 별시 성동구 선릉로 711', '서울특별시 성동구 천호대로 276', '서울특별 시 성동구 하왕십리1동', '서울특별시 성동구 하왕십리2동', '서울특별시 성동구 왕십리로30길 8', '서울특별시 성동구 왕십리로26길 2-1', '서울특별시 성동구 자동차시장1길 82', '서울특별시 성동구 자동차시장1길 90', '서울특별시 성동구 왕십리로24길 16-1', '서울특별시 성동구 중앙로265번길 30', '서울특별시 성동구 자동차시장1길  82', '서울특별시 성동구 용 답25다길 23', '서울특별시 성동구 마장로40길 11-1', '서울특별시 성동구 왕십리로31가길 10-2']
			# 개수: 16 => 직접 검색 ㄱㄱ
			no_results_doro.append(addr)
		else:
			first = response_json["results"]["juso"][0]
			doro_hjd_nm_dict[addr] = first["hemdNm"]
		i = idx+1

	except (requests.exceptions.ConnectionError,
		 	requests.exceptions.ConnectTimeout,
			requests.exceptions.ReadTimeout) as e:
		print(idx, addr, "처리 중 네트워크 에러 발생:", type(e).__name__)
		fail_doro.append(addr)
	except Exception as e:
		print(idx, addr, "처리 중 기타 예외 발생:", repr(e))
		fail_doro.append(addr)
	finally:
		time.sleep(0.1)


0 2개 이상의 결과가 존재함: 서울특별시 성동구 마장동
1 2개 이상의 결과가 존재함: 서울특별시 성동구 사근동
2 2개 이상의 결과가 존재함: 서울특별시 성동구 송정동
3 2개 이상의 결과가 존재함: 서울특별시 성동구 용답동
4 2개 이상의 결과가 존재함: 서울특별시 성동구 행당동
5 2개 이상의 결과가 존재함: 서울특별시 성동구 홍익동
6 2개 이상의 결과가 존재함: 서울특별시 성동구 행당1동
7 2개 이상의 결과가 존재함: 서울특별시 성동구 도선동
8 2개 이상의 결과가 존재함: 서울특별시 성동구 마장로
9 2개 이상의 결과가 존재함: 서울특별시 성동구 무학로
10 2개 이상의 결과가 존재함: 서울특별시 성동구 상원길
11 2개 이상의 결과가 존재함: 서울특별시 성동구 응봉동
12 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 9
13 2개 이상의 결과가 존재함: 서울특별시 성동구 무학로 8
14 2개 이상의 결과가 존재함: 서울특별시 성동구 상원6길
15 검색 결과가 존재하지 않음:  서울특별시 성동구 옥수2동
16 2개 이상의 결과가 존재함: 서울특별시 성동구 행당2동
17 2개 이상의 결과가 존재함: 서울특별시 성동구 금호동2가
18 2개 이상의 결과가 존재함: 서울특별시 성동구 금호동3가
20 2개 이상의 결과가 존재함: 서울특별시 성동구 금호로 62
23 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 11
24 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 17
25 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 25
26 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 29
27 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 31
28 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 33
29 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 35
30 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 39
31 2개 이상의 결과가 존재함: 서울특별시 성동구 마조로 7
32 2개 이상의 결

In [64]:
# send request 2 - for fail_doro
i = 0
for idx, addr in enumerate(fail_doro, start=i):
	params = {"confmKey" : SEARCH_CONFIRM_KEY,
		  "currentPage" : 1,
		   "countPerPage" : 10,
		   "keyword" : addr,
		   "resultType" : "json",
		   "addInfoYn" : "Y"
		}
	try:
		response_json = requests.get(url, params=params, timeout=(5, 10)).json()
		if len(response_json["results"]["juso"]) > 1:
			print(i, "2개 이상의 결과가 존재함:", addr) 
			# 서울특별시 성동구 상원2길 5를 검색하면 서울특별시 성동구 상원1길 5-5, 5-6, .. 이런 식으로 더 나옴.
			# 이런 경우 맨 첫 번째 결과가 가장 적합한 결과가 나오는 듯함.
			first = response_json["results"]["juso"][0]
			doro_hjd_nm_dict[addr] = first["hemdNm"]
		elif len(response_json["results"]["juso"]) < 1:
			print(i, "검색 결과가 존재하지 않음: ", addr)
			# 업데이트 되지 않은 도로명주소일 경우 이렇게 나옴.
			# 따로 모은 뒤에 그 양에 따라 어떻게 할지 정해볼 듯함.
			# 가능한 방법 1) 도로명주소에 직접 검색->크롤링
			# 가능한 방법 2) 법정동 -> 행정동/혹은 바로 행정동 코드로 시도해보기
			# 가능한 방법 3) 도로명 주소 업데이트 api 찾아보기
			# 		== 매핑되지 않은 도로명 ==
			# ['서울특별시 성동구 옥수2동', '서울특별시 성동구 매봉길 13', '서울특 별시 성동구 선릉로 711', '서울특별시 성동구 천호대로 276', '서울특별 시 성동구 하왕십리1동', '서울특별시 성동구 하왕십리2동', '서울특별시 성동구 왕십리로30길 8', '서울특별시 성동구 왕십리로26길 2-1', '서울특별시 성동구 자동차시장1길 82', '서울특별시 성동구 자동차시장1길 90', '서울특별시 성동구 왕십리로24길 16-1', '서울특별시 성동구 중앙로265번길 30', '서울특별시 성동구 자동차시장1길  82', '서울특별시 성동구 용 답25다길 23', '서울특별시 성동구 마장로40길 11-1', '서울특별시 성동구 왕십리로31가길 10-2']
			# 개수: 16 => 직접 검색 ㄱㄱ
			no_results_doro.append(addr)
		else:
			first = response_json["results"]["juso"][0]
			doro_hjd_nm_dict[addr] = first["hemdNm"]
		i = idx+1

	except (requests.exceptions.ConnectionError,
		 	requests.exceptions.ConnectTimeout,
			requests.exceptions.ReadTimeout) as e:
		print(idx, addr, "처리 중 네트워크 에러 발생:", type(e).__name__)
		fail_doro.append(addr)
	except Exception as e:
		print(idx, addr, "처리 중 기타 예외 발생:", repr(e))
		fail_doro.append(addr)
	finally:
		time.sleep(0.1)


1 2개 이상의 결과가 존재함: 서울특별시 성동구 아차산로7길 42


In [67]:
# debug
print("== 매핑되지 않은 도로명 ==")
print(no_results_doro)
print("개수:", len(no_results_doro))
print()
print("== 도로 - 행정동 매핑 결과 ==")
print(doro_hjd_nm_dict)
print("개수:", len(doro_hjd_nm_dict))
print()
print("개수 총 합:", len(no_results_doro)+len(doro_hjd_nm_dict),"\n", "전체 개수:", len(pd.unique(data1["MCT_BSE_AR"])))


== 매핑되지 않은 도로명 ==
['서울특별시 성동구 옥수2동', '서울특별시 성동구 매봉길 13', '서울특별시 성동구 선릉로 711', '서울특별시 성동구 천호대로 276', '서울특별시 성동구 하왕십리1동', '서울특별시 성동구 하왕십리2동', '서울특별시 성동구 왕십리로30길 8', '서울특별시 성동구 왕십리로26길 2-1', '서울특별시 성동구 자동차시장1길 82', '서울특별시 성동구 자동차시장1길 90', '서울특별시 성동구 왕십리로24길 16-1', '서울특별시 성동구 중앙로265번길 30', '서울특별시 성동구 용답25다길 23', '서울특별시 성동구 마장로40길 11-1', '서울특별시 성동구 왕십리로31가길 10-2']
개수: 15

== 도로 - 행정동 매핑 결과 ==
{'서울특별시 성동구 마장동': '서울특별시 성동구 마장동', '서울특별시 성동구 사근동': '서울특별시 성동구 사근동', '서울특별시 성동구 송정동': '서울특별시 성동구 송정동', '서울특별시 성동구 용답동': '서울특별시 성동구 용답동', '서울특별시 성동구 행당동': '서울특별시 성동구 행당제1동', '서울특별시 성동구 홍익동': '서울특별시 성동구 왕십리도선동', '서울특별시 성동구 행당1동': '서울특별시 성동구 행당제1동', '서울특별시 성동구 도선동': '서울특별시 성동구 왕십리도선동', '서울특별시 성동구 마장로': '서울특별시 성동구 왕십리도선동', '서울특별시 성동구 무학로': '서울특별시 성동구 왕십리도선동', '서울특별시 성동구 상원길': '서울특별시 성동구 성수1가제2동', '서울특별시 성동구 응봉동': '서울특별시 성동구 응봉동', '서울특별시 성동구 마조로 9': '서울특별시 성동구 사근동', '서울특별시 성동구 무학로 8': '서울특별시 성동구 왕십리도선동', '서울특별시 성동구 상원6길': '서울특별시 성동구 성수1가제2동', '서울특별시 성동구 행당2동': '서울특별시 성동구 행당제2동', '서울특별시 성동구 금호동2가': '서울특별시 성동

In [ ]:
# process no_results_doro
no_results_doro = ['서울특별시 성동구 옥수2동', '서울특별시 성동구 매봉길 13', '서울특별시 성동구 선릉로 711', '서울특별시 성동구 천호대로 276', '서울특별시 성동구 하왕십리1동', '서울특별시 성동구 하왕십리2동', '서울특별시 성동구 왕십리로30길 8', '서울특별시 성동구 왕십리로26길 2-1', '서울특별시 성동구 자동차시장1길 82', '서울특별시 성동구 자동차시장1길 90', '서울특별시 성동구 왕십리로24길 16-1', '서울특별시 성동구 중앙로265번길 30', '서울특별시 성동구 용답25다길 23', '서울특별시 성동구 마장로40길 11-1', '서울특별시 성동구 왕십리로31가길 10-2']
no_results_doro_hjd = pd.Series([
	'옥수동', 
	'옥수동', 
	'',
	'용답동',
	'',
	'',
	'도선동',
	'도선동',
	'용답동',
	'용답동',
	'도선동',
	'',
	'용답동',
	'마장동',
	'하왕십리동'
 ])

no_results_doro_hjd_dict = dict(zip(no_results_doro, "서울특별시 성동구 "+no_results_doro_hjd.astype(str)))
no_results_doro = [k for k, v in no_results_doro_hjd_dict.items() if v=="서울특별시 성동구 "]
no_results_doro_hjd_dict


{'서울특별시 성동구 옥수2동': '서울특별시 성동구 옥수동',
 '서울특별시 성동구 매봉길 13': '서울특별시 성동구 옥수동',
 '서울특별시 성동구 선릉로 711': '서울특별시 성동구 ',
 '서울특별시 성동구 천호대로 276': '서울특별시 성동구 용답동',
 '서울특별시 성동구 하왕십리1동': '서울특별시 성동구 ',
 '서울특별시 성동구 하왕십리2동': '서울특별시 성동구 ',
 '서울특별시 성동구 왕십리로30길 8': '서울특별시 성동구 도선동',
 '서울특별시 성동구 왕십리로26길 2-1': '서울특별시 성동구 도선동',
 '서울특별시 성동구 자동차시장1길 82': '서울특별시 성동구 용답동',
 '서울특별시 성동구 자동차시장1길 90': '서울특별시 성동구 용답동',
 '서울특별시 성동구 왕십리로24길 16-1': '서울특별시 성동구 도선동',
 '서울특별시 성동구 중앙로265번길 30': '서울특별시 성동구 ',
 '서울특별시 성동구 용답25다길 23': '서울특별시 성동구 용답동',
 '서울특별시 성동구 마장로40길 11-1': '서울특별시 성동구 마장동',
 '서울특별시 성동구 왕십리로31가길 10-2': '서울특별시 성동구 하왕십리동'}

In [94]:
doro_hjd_nm_dict.update(no_results_doro_hjd_dict)
doro_hjd_nm = pd.DataFrame.from_dict(doro_hjd_nm_dict, orient="index", columns=["행정동명"])
doro_hjd_nm = doro_hjd_nm.reset_index().rename(columns={"index":"도로명"})

hjd_nm_to_code = pd.DataFrame.from_dict(hjd_nm_to_code_dict, orient='index', columns=['행정동코드'])
hjd_nm_to_code = hjd_nm_to_code.reset_index().rename(columns={'index': '행정동명'})

hjd_nm_to_code["행정동명"] = "서울특별시 성동구 "+hjd_nm_to_code["행정동명"]
doro_hjd_nm["행정동명"] = doro_hjd_nm["행정동명"].str.replace("제", "")
doro_hjd_nm["행정동명"] = doro_hjd_nm["행정동명"].str.replace(".", "·")

doro_hjd_nm = doro_hjd_nm.merge(hjd_nm_to_code, how="left", on="행정동명")
data1_hjd = data1.merge(doro_hjd_nm[["도로명", "행정동코드"]], how="left", left_on="MCT_BSE_AR", right_on="도로명")
data1_hjd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4185 entries, 0 to 4184
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ENCODED_MCT         4185 non-null   object 
 1   MCT_BSE_AR          4185 non-null   object 
 2   MCT_NM              4185 non-null   object 
 3   MCT_BRD_NUM         542 non-null    object 
 4   MCT_SIGUNGU_NM      4185 non-null   object 
 5   HPSN_MCT_ZCD_NM     4185 non-null   object 
 6   HPSN_MCT_BZN_CD_NM  3138 non-null   object 
 7   ARE_D               4185 non-null   int64  
 8   MCT_ME_D            127 non-null    float64
 9   도로명                 4185 non-null   object 
 10  행정동코드               4176 non-null   float64
dtypes: float64(2), int64(1), object(8)
memory usage: 359.8+ KB


In [95]:
data1_hjd.drop("도로명", axis=1, inplace=True)

In [96]:
data1_hjd[data1_hjd["행정동코드"].isna()]

,ENCODED_MCT,MCT_BSE_AR,MCT_NM,MCT_BRD_NUM,MCT_SIGUNGU_NM,HPSN_MCT_ZCD_NM,HPSN_MCT_BZN_CD_NM,ARE_D,MCT_ME_D,행정동코드
233,15E95AF364,서울특별시 성동구 선릉로 711,잭슨******,NaN,서울 성동구,피자,NaN,20211025,NaN,NaN
576,4FECCA34D6,서울특별시 성동구 하왕십리1동,동순*,NaN,서울 성동구,중식당,NaN,20060425,NaN,NaN
577,25B2D74E6B,서울특별시 성동구 하왕십리2동,멕시***,치킨20,서울 성동구,치킨,NaN,20020130,NaN,NaN
578,B0E400DBAB,서울특별시 성동구 하왕십리2동,기분****,NaN,서울 성동구,한식-해물/생선,왕십리,20030124,NaN,NaN
949,EED0404EC2,서울특별시 성동구 왕십리로30길 8,자연*****,NaN,서울 성동구,농산물,NaN,20160504,NaN,NaN
1761,891AEAE4A4,서울특별시 성동구 왕십리로26길 2-1,안주****,NaN,서울 성동구,포장마차,NaN,20080731,NaN,NaN
2141,DA7C0F3255,서울특별시 성동구 왕십리로24길 16-1,성식******,NaN,서울 성동구,한식-육류/고기,NaN,20140822,NaN,NaN
2766,43C7FF8A46,서울특별시 성동구 중앙로265번길 30,은하***,NaN,서울 성동구,한식-해물/생선,행당,20191030,NaN,NaN
4153,DF67DE190B,서울특별시 성동구 왕십리로31가길 10-2,브렛********,NaN,서울 성동구,한식-단품요리일반,NaN,20190405,NaN,NaN


In [97]:
data1_hjd.dropna(subset=["행정동코드"], inplace=True)


In [104]:
data1_hjd.rename(columns={"행정동코드":"HJD_CODE"}, inplace=True)

In [105]:
data1_hjd.to_csv("data1_hjd.csv", index=False, encoding='utf-8-sig')

In [3]:
data1_hjd = pd.read_csv(r"C:\Users\USER\PycharmProjects\Big-Contest\data1_hjd.csv", encoding="utf-8-sig")

NameError: name 'pd' is not defined

In [1]:
data1_hjd["HJD_CODE"].value_counts()

NameError: name 'data1_hjd' is not defined